In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', 100)

import re
import string
import nltk

from textblob import TextBlob, Word
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# found a major class imbalance after doing work in merge_and_chop, so I'm going to fix that here
# also going to create a csv of roughly 20k instead of doing the train, test, and calidate csv files due to vocabulary problems

#this df is made from merge_and_chop using the dataset files at https://www.kaggle.com/forgemaster/steam-reviews-dataset?select=reviews-1-115.csv
#none of these files are stored in github
df = pd.read_csv("../../../../trimmed/combined_split_reviews.csv")
df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198322990549,620,Portal 2,"['Platformer', 'Dark Humor', 'Puzzle', 'Puzzle Platformer', 'First-Person', 'Story Rich', 'Femal...",whatley become breadly???? hwat??????,True,True
1,76561198207477984,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",Probably the game you will fall in love with. I did.,True,True
2,76561198157507512,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",This game has everything you could think of. \nAnywhere from chaotic drama to serene romance.\nT...,True,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3018279 entries, 0 to 3018278
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   steamid    int64 
 1   appid      int64 
 2   app_title  object
 3   app_tags   object
 4   review     object
 5   fps        bool  
 6   voted_up   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 120.9+ MB


In [4]:
df["voted_up"].value_counts()

True     2728583
False     289696
Name: voted_up, dtype: int64

In [5]:
df["voted_up"].value_counts(normalize=True)

True     0.904019
False    0.095981
Name: voted_up, dtype: float64

In [6]:
# above is where our class imbalance in the train, test, and validate splits comes from
# going to use the two below dataframes to take 10k random voted up and 10k random voted down
df_up = df.loc[df['voted_up'] == 1]
df_down = df.loc[df['voted_up'] == 0]

In [7]:
#getting 10k random voted up
df_up.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2728583 entries, 0 to 3018278
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   steamid    int64 
 1   appid      int64 
 2   app_title  object
 3   app_tags   object
 4   review     object
 5   fps        bool  
 6   voted_up   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 130.1+ MB


In [8]:
X_up = df_up.drop("voted_up", axis=1)
y_up = df_up["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_up, y_up, random_state=57, test_size=0.0035) #took some fiddling for test_size
display(X_test.shape, y_test.shape)

(9551, 6)

(9551,)

In [9]:
df_up_sample = pd.concat([X_test, y_test], axis=1)
df_up_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9551 entries, 2904230 to 2399743
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    9551 non-null   int64 
 1   appid      9551 non-null   int64 
 2   app_title  9551 non-null   object
 3   app_tags   9551 non-null   object
 4   review     9543 non-null   object
 5   fps        9551 non-null   bool  
 6   voted_up   9551 non-null   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 466.4+ KB


In [10]:
df_up_sample.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
2904230,76561198865552498,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",very good game,True,True
480087,76561197964533061,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",beause S,True,True
1483095,76561198290998839,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...","Russians everywhere, do not recommend",True,True


In [11]:
#looking good for the up votes, now the downvotes

X_down = df_down.drop("voted_up", axis=1)
y_down = df_down["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, random_state=57, test_size=0.035) #took some fiddling for test_size
display(X_test.shape, y_test.shape)

(10140, 6)

(10140,)

In [12]:
df_down_sample = pd.concat([X_test, y_test], axis=1)
df_down_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10140 entries, 1001414 to 1958918
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    10140 non-null  int64 
 1   appid      10140 non-null  int64 
 2   app_title  10140 non-null  object
 3   app_tags   10140 non-null  object
 4   review     10137 non-null  object
 5   fps        10140 non-null  bool  
 6   voted_up   10140 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 495.1+ KB


In [13]:
df_down_sample.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
1001414,76561198106930848,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",Nai-tupata igra na celiq svqt nqmam zaguba padam edna igra i mi pada ranka delete cs go!!!!!!!!!...,True,False
462268,76561198037219933,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...","I was new to this game, started to get better. Had a good round one time with a good friend of m...",True,False
1738746,76561198209141628,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",Nice job you made csgo horrible,True,False


In [14]:
#now to combine these two and then change voted_up to an int

final_df = pd.concat([df_up_sample, df_down_sample])
final_df.reset_index(drop=True, inplace=True)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19691 entries, 0 to 19690
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    19691 non-null  int64 
 1   appid      19691 non-null  int64 
 2   app_title  19691 non-null  object
 3   app_tags   19691 non-null  object
 4   review     19680 non-null  object
 5   fps        19691 non-null  bool  
 6   voted_up   19691 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 807.8+ KB


In [15]:
final_df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198865552498,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",very good game,True,True
1,76561197964533061,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",beause S,True,True
2,76561198290998839,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...","Russians everywhere, do not recommend",True,True


In [16]:
final_df["voted_up"] = final_df["voted_up"].astype(int)
final_df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198865552498,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",very good game,True,1
1,76561197964533061,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",beause S,True,1
2,76561198290998839,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...","Russians everywhere, do not recommend",True,1


In [17]:
final_df["voted_up"].value_counts(normalize=True)

0    0.514956
1    0.485044
Name: voted_up, dtype: float64

In [18]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19691 entries, 0 to 19690
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    19691 non-null  int64 
 1   appid      19691 non-null  int64 
 2   app_title  19691 non-null  object
 3   app_tags   19691 non-null  object
 4   review     19680 non-null  object
 5   fps        19691 non-null  bool  
 6   voted_up   19691 non-null  int32 
dtypes: bool(1), int32(1), int64(2), object(3)
memory usage: 865.5+ KB


In [20]:
final_df = final_df.dropna()

In [21]:
final_df.to_csv("../../../data/fps_main.csv", index=False)